In [2]:
# Librerias a usar
import pandas as pd
import numpy as np
import re

In [3]:
# Se carga el archivo a limpiar, en este caso es el "Reporte de Activos" el cual se obtiene desde el modulo de activo fijo, no confundir con el gran volumen ejercito, ya que los nombres de las columnas son diferentes entre los dos archivos, cabe considerar que el Sargento Mario Rojas y el PAC Francisco Caceres me confirmaron que los dos reportes son iguales
df_ori = pd.read_html('../Datos/XXEJE_FA___Reporte_de_Activos__010222.xls', header=0)[0]

In [12]:
# Aca se seleccionan las columnas de tipo obtejo, es decir, las que tienen 'str' en su contenido, o con contenido mixto, y se aplica la funcion 'str.strip()' para quitar los espacios en blanco al inicio y final de cada elemento
df_obj = df_ori.select_dtypes(['object'])
df_ori[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [16]:
# En esta celda se seleccionan las columnas que se ocuparan en el proceso de limpieza, se eliminan las filas dulicadas y se cambia al nombre a ciertas columnas
df = df_ori[['Numero Activo','MMU/PPU', 'Descripción', 'Marca', 'Modelo', 'Categoría SIGLE', 'UAC', 'UR', 'Costo Actual', 'Situación Operacional', 'Condición del Bien', 'Año Fabricación', 'Cód. CIMAGE', 'Tipo Patente', 'Rueda/Repuesto/Batería/Combustible', 'Capacidad Estanque LTS', 'Función']]
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
df.rename(columns={'Rueda/Repuesto/Batería/Combustible':'Combustible', 'Capacidad Estanque LTS':'Estanque'}, inplace=True)

In [17]:
# Continuo con el proceso de limpieza, elimino las filas que no posean informacion de patentes, esta sera mi PK, por lo que no puede estar vacia, y ademas elimino los valores que contienen un * ya que estos estan con patente duplicada, siendo los * patentes que no deberian estar en el sistema


# df.dropna(subset=['MMU/PPU'], inplace=True)
df['MMU/PPU'].fillna('SIN', inplace=True)
df_new = df[~df['MMU/PPU'].str.contains('\*')]

df = df_new.copy()
df['MMU/PPU'] = df['MMU/PPU'].str.replace('.','-', regex=True)
df['MMU/PPU'] = df['MMU/PPU'].str.replace(' ','', regex=True)

change_to_blanck = ['REP-', 'SINMATRICULA', 'S/M']
regular_expression = '[' + re.escape (''. join (change_to_blanck)) + ']'

df['MMU/PPU'] = df['MMU/PPU'].str.replace(regular_expression,'', regex=False)

# df['MMU/PPU'] = df['MMU/PPU'].str.replace('REP-','', regex=True)
# df['MMU/PPU'] = df['MMU/PPU'].str.replace('XW6996','XW-6996', regex=True)
# df['MMU/PPU'] = df['MMU/PPU'].str.replace('SINMATRICULA','', regex=True)
# df['MMU/PPU'] = df['MMU/PPU'].str.replace('S/M','', regex=True)

In [19]:
# Ahora busco las patentes que quedaron con formatos distintos al normal, y segun sea el caso, fuerzo a que sean normales

palabras = []
cambios = []
for i, n in enumerate(df['MMU/PPU']):
    match_w_n_w = re.findall("(([A-Z]+)([0-9]+)([A-Z]+))", n)
    match_n_w = re.findall("(([0-9]+)([A-Z]+))", n)
    match_w_n = re.findall("(([A-Z]+)([0-9]+))", n)
    if match_w_n_w:
        # palabras.append(match_w_n_w[0][0])
        palabras.append(n)
        b = []
        for (j, m) in list(enumerate(match_w_n_w[0]))[1:]:
            b.append(m)
        cambios.append('-'.join(b))
    elif match_n_w:
        # palabras.append(match_n_w[0][0])
        palabras.append(n)
        b = []
        for (j, m) in list(enumerate(match_n_w[0]))[1:]:
            b.append(m)
        cambios.append('-'.join(b))
    elif match_w_n:
        # palabras.append(match_w_n[0][0])
        palabras.append(n)
        b = []
        for (j, m) in list(enumerate(match_w_n[0]))[1:]:
            b.append(m)
        cambios.append('-'.join(b))
palabra_cambio = {}
palabra_cambio = dict(zip(palabras, cambios))

for i, n in enumerate(palabra_cambio):
    if len(n)<len(palabra_cambio[n])+1:
        df['MMU/PPU'] = df['MMU/PPU'].str.replace(n, palabra_cambio[n], regex=False)

In [20]:
# Se aplica el mismo conceto nuevamente para asegurar que queden con el mismo formato

palabras = []
cambios = []
for i, n in enumerate(df['MMU/PPU']):
    match_w_n_n = re.findall("(([A-Z]+)-([0-9]+)-([0-9]{1}$))", n)
    if match_w_n_n:
        palabras.append(n)
        b = []
        for (j, m) in list(enumerate(match_w_n_n[0]))[1:3]:
            b.append(m)
        cambios.append('-'.join(b))
palabra_cambio = {}
palabra_cambio = dict(zip(palabras, cambios))

for i, n in enumerate(palabra_cambio):
    # print(n, palabra_cambio[n])
    if len(n)>len(palabra_cambio[n]): 
        df['MMU/PPU'] = df['MMU/PPU'].str.replace(n, palabra_cambio[n], regex=False)

In [21]:
# Aca se realiza una arte importante de la estandarizacion, la cual es darle un nombre común a las distintas entradas, con esto se asegura que se tengan solo nombres comunes.

df['Descripción'] = df['Descripción'].str.replace('ALJIBE PARA AGUA','REMOLQUE ALJIBE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('MINI CARGADOR','MINICARGADOR', regex=False)
df['Descripción'] = df['Descripción'].str.replace('MONTACARGA CON HORQUILLA','MONTACARGA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('MOTO PARA NIEVE','MOTO DE NIEVE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('STATION WAGON','SUV', regex=False)
df['Descripción'] = df['Descripción'].str.replace('SEMIREMOLQUE CAMA BAJA','SEMIREMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('SEMIREMOLQUE EQUIPO LOGISTICO','SEMIREMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('SEMIREMOLQUE DE CARGA','SEMIREMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE DE CARGA','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE ESTANQUE','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE PARA USO EN NIEVE','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE EQUIPO LOGISTICO','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE ALJIBE','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE LIVIANO','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE TRIPLE (MOTOCICLETA)','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE TRIPLE','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE DE ARRASTRE ANIMAL (USO AGRICOLA)','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE MB 2300W','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE PESADO','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE CARGA (CUADRIMOTO)','REMOLQUE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('TRINEO DE ARRASTRE PARA MOTO DE NIEVE','TRINEO DE ARRASTRE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('TRINEO DE ARRASTRE ORION PARA MOTO DE NIEVE','TRINEO DE ARRASTRE', regex=False)
df['Descripción'] = df['Descripción'].str.replace('TRACTOR AGRICOLA','TRACTOR', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMIONETA DE TRASNPORTE PERSONAL','CAMIONETA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMIONETA TRASNPORTE PERSONAL','CAMIONETA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMIONETA TRASNPORTE SHELTER','CAMIONETA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMIÓN','CAMION', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMION TRASNPORTE SHELTER','CAMION', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMION TRANSPORTE SHELTER','CAMION', regex=False)
df['Descripción'] = df['Descripción'].str.replace('JEEP DE TRASNPORTE','JEEP', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMIONETA PARA ESTACION REPETIDORA','CAMIONETA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('REMOLQUE ALGIBE','REMOLQUE', regex=False)
# df['Descripción'] = df['Descripción'].str.replace(' CAMA BAJA ','CAMA BAJA', regex=False)
df['Descripción'] = df['Descripción'].str.replace('CAMION DE TRANSPORTE','CAMION', regex=False)

In [22]:
# Al igual que la celda anterior, aca se busca un nombre común a las distintas entradas
df['Marca'] = df['Marca'].str.replace('ALTO AMERICAN- LINCOLN','AMERICAN LINCOLN', regex=False)
df['Marca'] = df['Marca'].str.replace('AM GENERAL','AM GEN', regex=False)
df['Marca'] = df['Marca'].str.replace('AM GEN','AM GENERAL', regex=False)
df['Marca'] = df['Marca'].str.replace('ASIA MOTORS','ASIA', regex=False)
df['Marca'] = df['Marca'].str.replace('ASIA','ASIA MOTORS', regex=False)
df['Marca'] = df['Marca'].str.replace('BARRETT','BARRET', regex=False)
df['Marca'] = df['Marca'].str.replace('BARRET','BARRETT', regex=False)
df['Marca'] = df['Marca'].str.replace('BEIT ALFA TRAILER CO','BEIT ALFA', regex=False)
df['Marca'] = df['Marca'].str.replace('BEIT-ALFA TRAILER CO.','BEIT ALFA', regex=False)
df['Marca'] = df['Marca'].str.replace('BLUMHART','BLUMHARDT', regex=False)
df['Marca'] = df['Marca'].str.replace('BOBCAT INGERSOLL RAND','BOBCAT', regex=False)
df['Marca'] = df['Marca'].str.replace('BOBCAT MODELO S630 FULLJ','BOBCAT', regex=False)
df['Marca'] = df['Marca'].str.replace('BOMBARDIER BRP SKI DOO','BOMBARDIER', regex=False)
df['Marca'] = df['Marca'].str.replace('BOX/CB2.SANCHEZ/Balatas,Neumaticos,Correa distribucion y turbo con eje gastados','', regex=False)
df['Marca'] = df['Marca'].str.replace('BROCK HOUSE','BROCKHOUSE', regex=False)
df['Marca'] = df['Marca'].str.replace('CARDOEN','CARDOEN-MOWAG', regex=False)
df['Marca'] = df['Marca'].str.replace('CATARPILLAR','CATERPILLAR', regex=False)
df['Marca'] = df['Marca'].str.replace('CATERPILAR','CATERPILLAR', regex=False)
df['Marca'] = df['Marca'].str.replace('CATERPILLAR 130-G','CATERPILLAR', regex=False)
df['Marca'] = df['Marca'].str.replace('CATERPILLAR D6D','CATERPILLAR', regex=False)
df['Marca'] = df['Marca'].str.replace('COMAND CAR','COMMANDCAR', regex=False)
df['Marca'] = df['Marca'].str.replace('COMANDCAR','COMMANDCAR', regex=False)
df['Marca'] = df['Marca'].str.replace('COMMANCAR','COMMANDCAR', regex=False)
df['Marca'] = df['Marca'].str.replace('COMMAND CAR','COMMANDCAR', regex=False)
df['Marca'] = df['Marca'].str.replace('CUSTROM','CUSTOM', regex=False)
df['Marca'] = df['Marca'].str.replace('DAHIATSU','DAIHATSU', regex=False)
df['Marca'] = df['Marca'].str.replace('FDERAL MOTOR COMPANY','FEDERAL MOTOR COMPANY', regex=False)
df['Marca'] = df['Marca'].str.replace('FEDERAL MOTOR COMPANY','FEDERAL', regex=False)
df['Marca'] = df['Marca'].str.replace('FEDERAL','FEDERAL MOTOR COMPANY', regex=False)
df['Marca'] = df['Marca'].str.replace('FMC CORPORATION','FEDERAL MOTOR COMPANY', regex=False)
df['Marca'] = df['Marca'].str.replace('FORD RANGER','FORD', regex=False)
df['Marca'] = df['Marca'].str.replace('HYNDAI','HYUNDAI', regex=False)
df['Marca'] = df['Marca'].str.replace('HYUNDAY','HYUNDAI', regex=False)
df['Marca'] = df['Marca'].str.replace('HYUYNDAI','HYUNDAI', regex=False)
df['Marca'] = df['Marca'].str.replace('ELANTRA','HYUNDAI', regex=False)
df['Marca'] = df['Marca'].str.replace('INCA FREUHAUF','INCA', regex=False)
df['Marca'] = df['Marca'].str.replace('INCA FRUEHAUF','INCA', regex=False)
df['Marca'] = df['Marca'].str.replace('ISUSU','ISUZU', regex=False)
df['Marca'] = df['Marca'].str.replace('IZUSU','ISUZU', regex=False)
df['Marca'] = df['Marca'].str.replace('JOHNSON','TEST', regex=False)
df['Marca'] = df['Marca'].str.replace('JHON DEERE','JOHN', regex=False)
df['Marca'] = df['Marca'].str.replace('JOHN DEERE','JOHN', regex=False)
df['Marca'] = df['Marca'].str.replace('JHON DEER','JOHN', regex=False)
df['Marca'] = df['Marca'].str.replace('JOHN','JOHN DEERE', regex=False)
df['Marca'] = df['Marca'].str.replace('TEST','JOHNSON', regex=False)
df['Marca'] = df['Marca'].str.replace('KIA MOTORS','KIA', regex=False)
df['Marca'] = df['Marca'].str.replace('KIA SORENTO','KIA', regex=False)
df['Marca'] = df['Marca'].str.replace('KOMATSU PC 130-8','KOMATSU', regex=False)
df['Marca'] = df['Marca'].str.replace('M/BENZ','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MERC. BENZ','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MERCEDEZ BENZ','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MERCEDEZ-BENZ','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MERCEDES BENZ   ','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MERCEDES BENZ\t','MERCEDES BENZ', regex=False)
df['Marca'] = df['Marca'].str.replace('MAK','MACK', regex=False)
df['Marca'] = df['Marca'].str.replace('nissan','NISSAN', regex=False)
df['Marca'] = df['Marca'].str.replace('NETZER SERENI','NETZER', regex=False)
df['Marca'] = df['Marca'].str.replace('N.A','', regex=False)
df['Marca'] = df['Marca'].str.replace('N.A.','', regex=False)
df['Marca'] = df['Marca'].str.replace('SIM MARCA','', regex=False)
df['Marca'] = df['Marca'].str.replace('SIN INFORMACION','', regex=False)
df['Marca'] = df['Marca'].str.replace('SIN MARCA','', regex=False)
df['Marca'] = df['Marca'].str.replace('NO APLICA','', regex=False)
df['Marca'] = df['Marca'].str.replace('S/M','', regex=False)
df['Marca'] = df['Marca'].str.replace('POLARIS DEFENSE','POLARIS', regex=False)
df['Marca'] = df['Marca'].str.replace('POLARIS','DEFENSE', regex=False)
df['Marca'] = df['Marca'].str.replace('RANDOM','RANDON', regex=False)
df['Marca'] = df['Marca'].str.replace('TECNOVE SEGURITY','TECNOVE SECURITY', regex=False)
df['Marca'] = df['Marca'].str.replace('TOYOTA CAMRI','TOYOTA', regex=False)
df['Marca'] = df['Marca'].str.replace('TOYOTA LAND CRUISER II','TOYOTA', regex=False)
df['Marca'] = df['Marca'].str.replace('TOYOTA LAND CRUISER','TOYOTA', regex=False)
df['Marca'] = df['Marca'].str.replace('WILLYS BANTAM','WILLYS', regex=False)
df['Marca'] = df['Marca'].str.replace('SUZUKY','SUZUKI', regex=False)
df['Marca'] = df['Marca'].str.replace('ZUZUKI DR 350','SUZUKI', regex=False)
df['Marca'] = df['Marca'].str.replace('AUTOMOTIVE INDUSTRIES LTDA','AUTOMOTIVE INDUSTRIES LTD', regex=False)
df['Marca'] = df['Marca'].str.replace('AUTOMOVILE INDUST LTDA','AUTOMOTIVE INDUSTRIES LTD', regex=False)
df['Marca'] = df['Marca'].str.replace('BUICK SGM6531UAAA','BUICK', regex=False)
df['Marca'] = df['Marca'].str.replace('CHEVROLET DMAX','CHEVROLET', regex=False)
df['Marca'] = df['Marca'].str.replace('FORD AMBULANCIA','FORD', regex=False)
df['Marca'] = df['Marca'].str.replace('LIFTKING LK 630 R','LIFTKING', regex=False)
df['Marca'] = df['Marca'].str.replace('METALBERT LTDA.','METALBERT', regex=False)
df['Marca'] = df['Marca'].str.replace('METALVERT LTDA.','METALBERT', regex=False)
df['Marca'] = df['Marca'].str.replace('MITSIBISHI','MITSUBISHI', regex=False)
df['Marca'] = df['Marca'].str.replace('MITSUBISHI KATANA','MITSUBISHI', regex=False)
df['Marca'] = df['Marca'].str.replace('TECNOVE SECURITY','TECNOVE', regex=False)
df['Marca'] = df['Marca'].str.replace('.','', regex=False)

In [23]:
# Ahora se aplica una normalizacion de la informacion de la columna 'combustibles'
df['Combustible']=np.where(df['Combustible'].str.contains('DIESEL'), 'DIESEL', df['Combustible'])
df['Combustible']=np.where(df['Combustible'].str.contains('BENCI'), 'BENCINERO', df['Combustible'])
df['Combustible']=np.where(df['Combustible'].str.contains('NO APLICA'), 'NO APLICA', df['Combustible'])
df['Combustible']=np.where((df['Combustible'].str.contains('NO APLICA')) | (df['Combustible'].str.contains('DIESEL')) | (df['Combustible'].str.contains('BENCI')), df['Combustible'], 'NO APLICA')

In [24]:
# Finalmente se quitan las letras a la columna estanque, y se guarda todo con formato numerico
df['Estanque'] = df['Estanque'].str.extract('(\d+)', expand=False)
df['Estanque'] = pd.to_numeric(df['Estanque'])

In [25]:
# Finalmente se guarda un archivo con los datos ya estandarizados
df.to_csv('Clean_Bd_Vehic.csv', encoding='latin-1', index=False)